In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install accelerate -U
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
nsmc = load_dataset("nsmc")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
from transformers import BertTokenizer
pretrained_model_name = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["document"],truncation=True)

In [ ]:
dataset_encoded = nsmc.map(tokenize, batched=True)

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions,axis=1)
    return accuracy.compute(predictions=predictions,references=labels)

In [ ]:
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2,id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

In [ ]:
from transformers import TrainingArguments
batch_size = 64
training_args = TrainingArguments(
    "my_doccls_korean_model",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded["train"],
    eval_dataset=dataset_encoded["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/chunwoolee0/my_doccls_korean_model into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267000,0.248223,0.898720
2,0.175100,0.252315,0.902420
3,0.110800,0.294226,0.903720


TrainOutput(global_step=7032, training_loss=0.19521064888495226, metrics={'train_runtime': 5611.7277, 'train_samples_per_second': 80.189, 'train_steps_per_second': 1.253, 'total_flos': 1.61488110226416e+16, 'train_loss': 0.19521064888495226, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/416M [00:00<?, ?B/s]

Upload file runs/Jul13_16-51-48_840b8892d712/events.out.tfevents.1689267151.840b8892d712.697.0:   0%|         …

To https://huggingface.co/chunwoolee0/my_doccls_korean_model
   5fc7889..87995e0  main -> main

   5fc7889..87995e0  main -> main

To https://huggingface.co/chunwoolee0/my_doccls_korean_model
   87995e0..485f698  main -> main

   87995e0..485f698  main -> main



'https://huggingface.co/chunwoolee0/my_doccls_korean_model/commit/87995e020a720e7ffeb4e085cc2b054e0cd38b31'

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="chunwoolee0/my_doccls_korean_model")
text = '왕 짜증난다'
classifier(text)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'negative', 'score': 0.9889811277389526}]

In [ ]:
text = '그런대로 무난하다'
classifier(text)

[{'label': 'negative', 'score': 0.5668719410896301}]

In [ ]:
text = '잘 만들어졌다'
classifier(text)

[{'label': 'positive', 'score': 0.9369205832481384}]

In [ ]:
text = '엉망이다'
classifier(text)

[{'label': 'negative', 'score': 0.9969717264175415}]